In [ ]:
%reload_ext autoreload
%autoreload
%reload_ext dotenv
%dotenv
%run C:/Users/zfriant/Documents/GitHub/pydref/pydref.py
pydref = Pydref()

import pandas as pd, requests, re, string, time, urllib3,  os, dotenv
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# déclaration variable année de remise du prix
INPUT_FILE = "datas-source/etoiles_"
an = 2022
edition = 10
PATH = "C:/Users/zfriant/Documents/OneDrive/PCRI/Etoiles/"

In [ ]:
# export de la dernière année à traiter: etoiles_2020.xlsx
etoiles = pd.DataFrame(pd.read_excel(PATH + INPUT_FILE + str(an) + ".xlsx", na_values=" ", sheet_name=0, keep_default_na=False,
                                 dtype={'id_struct': str} ))

In [ ]:
etoiles.columns.values.tolist()

In [ ]:
etoiles.columns = etoiles.columns.str.lower()
etoiles.columns = etoiles.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
etoiles.columns.values.tolist()

l = ['prenom', 'nom']
for i in l:
    etoiles[i] = etoiles[i].apply(lambda x: str(x).replace(u'\xa0', u'').strip())
    
# renommage des vars à utiliser dans les traitements
etoiles = etoiles.rename(columns={'project_nbr':"project_id", "genre":"sexe"})
etoiles.columns.values.tolist()

etoiles = etoiles.loc[:, ["project_id", "prenom", "nom", "id_struct", "sexe", 'mention']]
etoiles = etoiles.assign(person =  etoiles["prenom"] +" "+ etoiles["nom"])

# etoiles = etoiles.assign(type_de_prix = "Etoiles de l'Europe")
etoiles = etoiles.assign(type_de_prix="Etoiles de l'Europe", edition=str(an), person=etoiles["prenom"] +" "+ etoiles["nom"])

In [ ]:
et = etoiles.to_dict(orient = "records")

In [ ]:
# # fonction de match des personnes pour récupérer le idref
counter = 0
for e in et:
    try:
        response = pydref.identify(e['person'])
    except:
        print(f"erreur pour {e['person']} on attend 5s.")
        time.sleep(5)
        continue
    if not response.get('idref'):
        print(f"NO idref:{e['person']}" )
    else:
        counter += 1
        print(counter)
        e['idref'] = response.get('idref')
    
        
#     del(e['person'])
    

In [ ]:
# match avec idref par le nom et prénom, update avec le idref detecté sinon récupération du idrefold

for e in et:     
#     e["idref"] = persons_matcher(e)
#     if e["idref"] is None and e["idrefold"]:
#         e["idref"] = e["idrefold"]
#     del(e["idrefold"])
    if not e["idref"] is None:
        e["idref"] = e["idref"].strip("idref")
        e["idref_lien"] = "http://www.idref.fr/" + e["idref"]
    else:
        e["idref"] = ""
        e["idref_lien"] = ""
    
    if e['sexe']:
        e['sexe'] = e['sexe'].lower()
        if re.search(r'^h', e['sexe']):
            e['sexe'] = 'homme'
            e['sexe_code'] = '1'
        if re.search(r'^f', e['sexe']):
            e['sexe'] = 'femme'
            e['sexe_code'] = '2'
        e['sexe'] = e['sexe'].capitalize()



In [ ]:
project_dict = {}
for e in et:
    list_to_change=['prenom', 'nom', 'idref', 'idref_lien', 'sexe_code', 'sexe']
    project_id = e['project_id']
    if project_id not in project_dict:
        new_elt = {'project_id': project_id}
        for col in ['mention', 'edition', 'type_de_prix', 'id_struct']:
            new_elt[col] = e[col]
        for col in list_to_change:
            new_elt[col] = []
    else:
        new_elt = project_dict[project_id]
    for col in list_to_change:    
        new_elt[col].append(e[col])
    project_dict[project_id] = new_elt

for e in project_dict:
    elt = project_dict[e]
    for f in elt:
        if isinstance(elt[f], list):
            project_dict[e][f] = ';'.join(elt[f])
            
tmp=pd.DataFrame(project_dict.values())
tmp=tmp.to_dict(orient="records")

In [ ]:
# fonction de match pour les organisations: lien avec id_struct rnsr, siren,...

def struct_matcher(data):
    print(data.get('id_struct'))
    url = f"http://185.161.45.213/organizations/organizations/_match?id={data.get('id_struct')}"
    rq = requests.get(url=url, headers={'accept':'application/json', 'Authorization': os.environ.get('token')})
    response = rq.json()
    if response.get("hits") == 1:
        result = response.get("data")[0].get("id")
        return result

In [ ]:
# match pour récupérer le id_scanr des structures et créer un url scanr
for e in tmp: 
    e["structure_identifiant"] = struct_matcher(e)
    if e["structure_identifiant"]:
        e["structure_lien_scanr"] = "https://scanr.enseignementsup-recherche.gouv.fr/structure/" + e.get("structure_identifiant")
    if e["structure_identifiant"] is None and e["id_struct"]:
        e["structure_identifiant"] = e["id_struct"]
    del(e["id_struct"])    

In [ ]:
# fonction récupérer les infos des structures libell, sigle, com_code, geoloc...

def struct_info(data):
    url_match = "http://185.161.45.213/organizations/scanr/"
    if data.get("structure_identifiant"):
        url = url_match + data.get("structure_identifiant")
        rq = requests.get(url = url,  headers={'accept':'application/json', 'Authorization': os.environ.get('token')})
        response = {
            "structure_libelle": rq.json().get("label", {}).get("default"),
            "structure_sigle": rq.json().get("acronym", {}).get("fr"),
#             "structure_nature": rq.json().get("kind")
        }
        if rq.json().get("address"):
            resp_temp = {
            "commune_code": rq.json().get("address")[0].get("citycode"),
            "lat": str(rq.json().get("address")[0].get("gps", {}).get("lat")),
            "lon": str(rq.json().get("address")[0].get("gps", {}).get("lon"))
        }
            response.update(resp_temp)
        return response

In [ ]:
# lancement fonction recup infos des structures et remplace les sauts de ligne par un espace (pas sur que ça marche !)
for e in tmp:
    if struct_info(e) is not None:
        e.update(struct_info(e))
    if e.get("lat"):
        e["geolocalisation"] = e.pop("lat") + ", " + e.pop("lon")

In [ ]:
# fonction récupération des codes de geoloc region, dep, unite urbaine...
def geocod(data):
    url_match = "http://185.161.45.213/datastore/geocodes/"
    if data.get("commune_code"):
        url = url_match + data.get("commune_code")
        rq = requests.get(url = url,  headers={'accept':'application/json', 'Authorization': os.environ.get('token')})
        response = {
            "commune_nom": rq.json().get("com_nom"),
            "unite_urbaine_code": rq.json().get("uu_id"),
            "unite_urbaine_nom": rq.json().get("uucr_nom"),
            "departement_code": rq.json().get("dep_id"),
            "departement_nom": rq.json().get("dep_nom"),
            "region_code": rq.json().get("reg_id"),
            "region_nom": rq.json().get("reg_nom")
        }
        return response

In [ ]:
for e in tmp:
    if geocod(e):       
        e.update(geocod(e))

In [ ]:
def project_info(data):
    url_match = "http://185.161.45.213/projects/projects/"
    id = str(data.get("project_id"))
    if id:
        url = url_match + id
        rq = requests.get(url = url,  headers={'accept':'application/json', 'Authorization': os.environ.get('token')})
        if rq.status_code == 200:
            if rq.json().get("type") == "H2020":
                response = {
                    "financement_type": rq.json().get("action")[0]['code'],
                    "financement_name": rq.json().get("action")[0]['name'],
                    "programme_code": rq.json().get("priorities")[1]['code'],
                    "programme_nom": rq.json().get("priorities")[1]['name'],
                    "appel_a_projet_code": rq.json().get("call_code"),
                    "projet_acronyme": rq.json().get("acronym"),
                    "projet_titre": rq.json().get("name").get("en"),
                    "projet_resume": rq.json().get("description").get("en", "")
                }

                return response
            
            if rq.json().get("type") == "FP7":
                response = {
                    "financement_type": rq.json().get("action")[0]['code'],
                    "financement_name": rq.json().get("action")[0]['name'],
                    "programme_code": rq.json().get("priorities")[1]['code'],
                    "programme_nom": rq.json().get("priorities")[1]['name'],
                    "appel_a_projet_code": rq.json().get("call_name"),
                    "projet_acronyme": rq.json().get("acronym"),
                    "projet_titre": rq.json().get("name").get("en"),
                    "projet_resume": rq.json().get("description").get("en", "")
                }

                return response

In [ ]:
for e in tmp: 
    e["projet_lien_scanr"] = "https://scanr.enseignementsup-recherche.gouv.fr/project/" + str(e["project_id"])
    e["projet_lien_cordis"] = "https://cordis.europa.eu/project/id/" + str(e["project_id"])
    
    if project_info(e):  
        e.update(project_info(e))

    l = ["projet_resume", "projet_titre"]
    for i in l:
        if e.get(i):
            e[i] = re.sub('[\t\n\r]', ' ', e[i])
            e[i] = re.sub('\s+', ' ', e[i])           

In [ ]:
full=pd.DataFrame(pd.read_excel(PATH + "etoiles_europe_full.xlsx", na_values=" ", sheet_name=0, keep_default_na=False ))

In [ ]:
export = pd.DataFrame.from_dict(tmp, orient='columns')
export = pd.concat([full, export], ignore_index=True)

In [ ]:
export = export[['type_de_prix', 'edition', 'mention', 'prenom', 'nom', 'sexe_code', 'sexe', 'idref', 
            'projet_acronyme', 'projet_titre', 'appel_a_projet_code', 'thematique', 
            'programme_code', 'programme_nom', 'financement_type', 'financement_name', 'project_id', 'projet_resume',
            'structure_identifiant', 'structure_libelle', 'structure_sigle', 
            'commune_code', 'commune_nom', 'unite_urbaine_code', 'unite_urbaine_nom', 'departement_code', 'departement_nom',
            'region_code', 'region_nom', 'geolocalisation', 'idref_lien', 'projet_lien_scanr', 'projet_lien_cordis',
            'structure_lien_scanr']]

In [ ]:
export.to_csv(PATH + "etoiles_europe_2013-" + str(an) + ".csv", sep=";", 
              encoding="ANSI", na_rep="", index=False, line_terminator='\r')

'''VERIFIER dans le fichier CSV qu\'il ne manque pas des infos existantes comme les idref non trouvés et sauver ce fichier
à nouveau dans le dossier OPENDATA sous le nom fr-esr-etoile-de-l-europe.csv'''

# export.to_csv(PATH + "open data/fr-esr-etoile-de-l-europe.csv", sep=";", 
#               encoding="ANSI", na_rep="", index=False, line_terminator='\r')